In [6]:
from bson import ObjectId
# from jupyter_plotly_dash import jupyterdash
from jupyter_dash import JupyterDash

from dash import html
import dash_table
from dash.dependencies import Input, Output

import pandas as pd
import dash_leaflet as dl

###########################
# data manipulation / model
###########################
# change animal_shelter and animalshelter to match your crud python module file name and class name
from animal_shelter import AnimalShelter

# fix me update with your username and password and crud python module name
username = "aacuser"
password = "0925"
shelter = AnimalShelter(username, password, port=27017, auth_source='admin')

# readall is the method name from animalshelter class and returns a cursor object
# update the next line to match your method name from your crud python module class
df = pd.DataFrame.from_records(shelter.read({})).drop(columns=['_id'])

#########################
# dashboard layout / view
#########################
app = JupyterDash('dash datatable only')

app.layout = html.Div([
    html.Center(html.B(html.H1('fabian rodriguez snhu cs-340 dashboard'))),
    html.Hr(),
    # button location at top
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Button(id='submit-button-one', n_clicks=0,
                             children='cats'),
                 html.Button(id='submit-button-two', n_clicks=0,
                             children='dogs')
             ]),
    dash_table.DataTable(
        id='datatable',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i
            in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),


])


#############################################
# interaction between components / controller
#############################################
@app.callback(
    Output('map-id', "children"),
    [Input('datatable', "derived_viewport_data")],
)
def update_map(viewData, **kwargs):
    dff = pd.DataFrame.from_dict(viewData)
    # drop the _id column
    if '_id' in dff.columns:
        dff = dff.drop(columns=['_id'])

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75, -97.48], children=[
                    dl.Tooltip(dff.iloc[0, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[1, 9])
                    ])
                ])
            ]),
    ]


@app.callback(Output('datatable', "data"),
              [
                  Input('submit-button-one', 'n_clicks'),
                  Input('submit-button-two', 'n_clicks'),
              ],
              )
def on_click(bt1, bt2, **kwargs):
    # start case
    if int(bt1) == 0 and int(bt2) == 0:
        df = pd.DataFrame.from_records(shelter.read({})).drop(columns=['_id'])
    # use higher number of button clicks to determine filter type, can you think of a better way ? ....
    elif int(bt1) > int(bt2):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Cat"}))).drop(columns=['_id'])
    elif int(bt2) > int(bt1):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Dog"}))).drop(columns=['_id'])

    return df.to_dict('records')

app

Connected to MongoDB
